In [1]:
result = """{
"locations": [
{
"name": "USA",
"context": "Ort des Geschehens",
"coordinates": {
"lat": 37.0902,
"long": -95.7129
}
},
{
"name": "Italien",
"context": "Herkunftsland der Verurteilten",
"coordinates": {
"lat": 41.8719,
"long": 12.5674
}
},
{
"name": "weltweit",
"context": "Ort der Proteste",
"coordinates": {}
}
],
"time": {
"description": "Die Geschichte spielt von 1917 bis 1927",
"start_year": 1917,
"end_year": 1927
},
"categories": [
{
"category_name": "Justizirrtum",
"description": "Die Geschichte handelt von einem möglichen Justizirrtum und der Verurteilung unschuldiger Personen."
},
{
"category_name": "Politik",
"description": "Die Geschichte beschäftigt sich mit politischen Entwicklungen und Ereignissen."
},
{
"category_name": "Migration",
"description": "Die Geschichte handelt von Einwanderern und ihren Erfahrungen in einem neuen Land."
},
{
"category_name": "Protest",
"description": "Die Geschichte beschreibt Proteste und Demonstrationen gegen ein Urteil."
}
]
}"""

In [15]:
from pydantic import BaseModel, Field

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

from pydantic import BaseModel, Field, validator
from typing import List, Optional
from tqdm import tqdm


response_json_format = """{
"locations": [
{
"name":
"context":
"coordinates": {
"lat":
"long":
}
}
],
"time": {
"description":
"start_year":
"end_year":
},
"categories": [
{
"category_name":
"description":
}
]
}"""


class Coordinate(BaseModel):
    lat: float = Field(..., example=52.520008)
    long: float = Field(..., example=13.404954)


class Location(BaseModel):
    name: str = Field(..., example="Berlin")
    context: Optional[str] = Field(..., example="Hauptstadt von Deutschland")
    coordinates: Optional[Coordinate] = Field(..., example={
        "lat": 52.520008, "long": 13.404954})
    
    @validator('coordinates', pre=True)
    def empty_dict_to_none(cls, v):
        if v == {}:
            return None
        return v


class Time(BaseModel):
    description: str = Field(...,
                             example="Die Geschichte spielt im 19. Jahrhundert.")
    start_year: Optional[float] = Field(..., example=1800)
    end_year: Optional[float] = Field(..., example=1900)


class Category(BaseModel):
    category_name: str = Field(..., example="Krieg")
    description: str = Field(
        ..., example="Der Hauptakteur ist ein Soldat im 2. Weltkrieg.")


class Response(BaseModel):
    locations: List[Location] = Field(
        description="Eine Liste von Orten, die in der Geschichte vorkommen.")
    time: Time = Field(description="Die Zeit, in der die Geschichte spielt.")
    categories: List[Category] = Field(
        description="Eine Liste von Kategorien, in die die Geschichte eingeordnet werden kann.")


def get_response_parser():
    parser = PydanticOutputParser(pydantic_object=Response)
    output_fixer = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

    return parser

In [16]:
parser = get_response_parser()

parser.parse(result)

Response(locations=[Location(name='USA', context='Ort des Geschehens', coordinates=Coordinate(lat=37.0902, long=-95.7129)), Location(name='Italien', context='Herkunftsland der Verurteilten', coordinates=Coordinate(lat=41.8719, long=12.5674)), Location(name='weltweit', context='Ort der Proteste', coordinates=None)], time=Time(description='Die Geschichte spielt von 1917 bis 1927', start_year=1917.0, end_year=1927.0), categories=[Category(category_name='Justizirrtum', description='Die Geschichte handelt von einem möglichen Justizirrtum und der Verurteilung unschuldiger Personen.'), Category(category_name='Politik', description='Die Geschichte beschäftigt sich mit politischen Entwicklungen und Ereignissen.'), Category(category_name='Migration', description='Die Geschichte handelt von Einwanderern und ihren Erfahrungen in einem neuen Land.'), Category(category_name='Protest', description='Die Geschichte beschreibt Proteste und Demonstrationen gegen ein Urteil.')])